In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import opendatasets as od
import pandas as pd
import os 
import random
import torch
import glob
from torch.utils.data import random_split
import torchvision.models as models

torch.manual_seed(1)

In [ ]:
import torchvision.datasets as datasets

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

geoguessr_data = os.getcwd() + '/geoguessr_data'

train_path = geoguessr_data+'/train'
test_path = geoguessr_data+'/test'
valid_path = geoguessr_data+'/val'


train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
val_dataset = datasets.ImageFolder(root=valid_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=27, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=27, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=27, shuffle=False)

# Get the class names from the dataset
class_names = train_dataset.classes

In [ ]:
print(class_names)

['Brazil', 'France', 'Japan', 'United Kingdom', 'United States']


In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [ ]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    for imgs, labels in data_loader:
        
        if torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()

        output = model(imgs)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
def train(model, train_loader, val_loader, batch_size=27, learning_rate = 0.001, num_epochs=30):
    ########################################################################
    # Fixed PyTorch random seed for reproducible result
    torch.manual_seed(1000)
    ########################################################################
    # Define the Loss function and optimizer
    # The loss function will be Cross Entropy Loss and the optimizer will be Adam
    # These two were selected because:
    # Cross Entropy Loss - Penalizes the model more heavily when it makes larger errors, which is desirable for image classification
    # Adam - Converges faster than other optimization algorithms, which is important for large-scale image classification
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    ########################################################################
    # Set up some numpy arrays to store the training/test accuracy
    train_acc = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    ########################################################################
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    start_time = time.time()
    print ("Training Started...")
    n = 0 # Iteration number
    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_train_err = 0.0
        total_images = 0
        for imgs, labels in iter(train_loader):
            
            if torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()

            # Forward pass, backward pass, and optimize
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            # Zero the parameter gradients
            optimizer.zero_grad()
            n += 1          
        
        # Get accuracy
        train_acc[epoch] = get_accuracy(model, train_loader)
        val_acc[epoch] = get_accuracy(model, val_loader)

        print(("Epoch {}: Train acc: {} |" + "Validation acc: {}").format(epoch, train_acc[epoch], val_acc[epoch]))

        model_path = get_model_name(model.name, batch_size, learning_rate, epoch)
        torch.save(model.state_dict(), model_path)
            
    epochs = np.arange(1, num_epochs + 1)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    
    return train_acc, val_acc, epochs

In [ ]:
def plot_curve(acc, epochs, curve_name):
  plt.plot(epochs, acc)
  plt.title(curve_name + " Curve (Default Parameters)")
  plt.xlabel("Epochs")
  plt.ylabel(curve_name + " Accuracy")
  plt.show()
  return

In [ ]:
class MobileNet(nn.Module):
    def __init__(self, num_classes=5):
        super(MobileNet, self).__init__()
        self.name = "mobilenet"
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.features = mobilenet.features
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1280, out_features=num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])  # Global Average Pooling
        x = self.classifier(x)
        return x

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.name = "cnn_model"
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 5)
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

class IntegratedModel(nn.Module):
    def __init__(self, num_classes=5):
        super(IntegratedModel, self).__init__()
        self.name = "integrated_model"
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.cnn_model = CNNModel()
        self.fc = nn.Linear(1280 + 5, num_classes) # 1280 is the output size of mobilenet's features and 5 is the output size of CNNModel's classifier
        
    def forward(self, x):
        mobilenet_features = self.mobilenet.features(x)
        mobilenet_features = mobilenet_features.mean([2, 3]) # Global Average Pooling
        cnn_output = self.cnn_model(x)
        combined_output = torch.cat((mobilenet_features, cnn_output), 1)
        output = self.fc(combined_output)
        return output

In [ ]:
model_integrated = IntegratedModel()
if torch.cuda.is_available():
    model_integrated.cuda()
train_acc_integrated, val_acc_integrated, epochs_integrated = train(model_integrated, train_loader, val_loader, batch_size=27, learning_rate = 0.0005, num_epochs=20)
plot_curve(train_acc_integrated, epochs_integrated, "Training")
plot_curve(val_acc_integrated, epochs_integrated, "Validation")

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training Started...
Epoch 0: Train acc: 0.7535 |Validation acc: 0.6605
Epoch 1: Train acc: 0.8465 |Validation acc: 0.7195
Epoch 2: Train acc: 0.8768333333333334 |Validation acc: 0.717
Epoch 3: Train acc: 0.9078333333333334 |Validation acc: 0.737
Epoch 4: Train acc: 0.9316666666666666 |Validation acc: 0.7585
Epoch 5: Train acc: 0.9303333333333333 |Validation acc: 0.7375
Epoch 6: Train acc: 0.9361666666666667 |Validation acc: 0.7335
Epoch 7: Train acc: 0.961 |Validation acc: 0.76
Epoch 8: Train acc: 0.9558333333333333 |Validation acc: 0.7425
Epoch 9: Train acc: 0.9333333333333333 |Validation acc: 0.721
Epoch 10: Train acc: 0.976 |Validation acc: 0.75
Epoch 11: Train acc: 0.9773333333333334 |Validation acc: 0.757
Epoch 12: Train acc: 0.9718333333333333 |Validation acc: 0.7525
Epoch 13: Train acc: 0.9546666666666667 |Validation acc: 0.7325
Epoch 14: Train acc: 0.9673333333333334 |Validation acc: 0.745
Epoch 15: Train acc: 0.9795 |Validation acc: 0.7435
Epoch 16: Train acc: 0.97833333333333

KeyboardInterrupt: ignored

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.name = "cnn_model"
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 5)
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

class IntegratedModel2(nn.Module):
    def __init__(self, num_classes=5):
        super(IntegratedModel2, self).__init__()
        self.name = "integrated_model2"
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.features = nn.Sequential(*list(resnet.children())[:-1])
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.cnn_model = CNNModel()
        self.fc = nn.Linear(512 + 1280 + 5, num_classes) # 1280 is the output size of mobilenet's features and 5 is the output size of CNNModel's classifier
        
    def forward(self, x):
        resnet_features = self.resnet.features(x)
        resnet_features = resnet_features.view(x.size(0), -1)
        mobilenet_features = self.mobilenet.features(x)
        mobilenet_features = mobilenet_features.mean([2, 3]) # Global Average Pooling
        cnn_output = self.cnn_model(x)
        combined_output = torch.cat((resnet_features, mobilenet_features, cnn_output), 1)
        output = self.fc(combined_output)
        return output

In [ ]:
model_integrated2 = IntegratedModel2()
if torch.cuda.is_available():
    model_integrated2.cuda()
train_acc_integrated2, val_acc_integrated2, epochs_integrated2 = train(model_integrated2, train_loader, val_loader, batch_size=27, learning_rate = 0.0005, num_epochs=20)
plot_curve(train_acc_integrated2, epochs_integrated2, "Training")
plot_curve(val_acc_integrated2, epochs_integrated2, "Validation")

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the mos

Training Started...
Epoch 0: Train acc: 0.7451666666666666 |Validation acc: 0.6575
Epoch 1: Train acc: 0.8481666666666666 |Validation acc: 0.732
Epoch 2: Train acc: 0.8795 |Validation acc: 0.7275
Epoch 3: Train acc: 0.9068333333333334 |Validation acc: 0.74
Epoch 4: Train acc: 0.9036666666666666 |Validation acc: 0.736
Epoch 5: Train acc: 0.9055 |Validation acc: 0.7075
Epoch 6: Train acc: 0.9476666666666667 |Validation acc: 0.7385
Epoch 7: Train acc: 0.949 |Validation acc: 0.7545
Epoch 8: Train acc: 0.9716666666666667 |Validation acc: 0.756
Epoch 9: Train acc: 0.9556666666666667 |Validation acc: 0.728
Epoch 10: Train acc: 0.9576666666666667 |Validation acc: 0.7495
Epoch 11: Train acc: 0.9673333333333334 |Validation acc: 0.745
Epoch 12: Train acc: 0.9845 |Validation acc: 0.7515
Epoch 13: Train acc: 0.968 |Validation acc: 0.7455


KeyboardInterrupt: ignored

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.name = "cnn_model"
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 5)
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=5):
        super(EnsembleModel, self).__init__()
        self.name = "ensemble_model"

        self.resnet = models.resnet18(pretrained=True)
        self.resnet_features = nn.Sequential(*list(self.resnet.children())[:-1])

        self.inception_v3 = models.inception_v3(pretrained=True)
        self.inception_features = nn.Sequential(
            self.inception_v3.Conv2d_1a_3x3,
            self.inception_v3.Conv2d_2a_3x3,
            self.inception_v3.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2),
            self.inception_v3.Conv2d_3b_1x1,
            self.inception_v3.Conv2d_4a_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2),
            self.inception_v3.Mixed_5b,
            self.inception_v3.Mixed_5c,
            self.inception_v3.Mixed_5d,
            self.inception_v3.Mixed_6a,
            self.inception_v3.Mixed_6b,
            self.inception_v3.Mixed_6c,
            self.inception_v3.Mixed_6d,
            self.inception_v3.Mixed_6e,
            self.inception_v3.Mixed_7a,
            self.inception_v3.Mixed_7b,
            self.inception_v3.Mixed_7c,
        )
        self.inception_avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.mobilenet_features = self.mobilenet.features

        self.custom_cnn = CNNModel()

        self.fc = nn.Linear(512 + 2048 + 1280 + 5, num_classes)
        
    def forward(self, x):
        resnet_out = self.resnet_features(x)
        resnet_out = resnet_out.view(resnet_out.size(0), -1)

        inception_out = self.inception_features(x)
        inception_out = self.inception_avgpool(inception_out)
        inception_out = torch.flatten(inception_out, 1)

        mobilenet_out = self.mobilenet_features(x)
        mobilenet_out = mobilenet_out.mean([2, 3])

        custom_out = self.custom_cnn(x)
        
        out = torch.cat((resnet_out, inception_out, mobilenet_out, custom_out), dim=1) # Stacking Ensemble Beta
        # out = torch.mean(torch.stack(resnet_out, inception_out, mobilenet_out, custom_out), dim=0) # Averaging Ensemble Beta
        # Weighted Ensemble?
        out = self.fc(out)
        
        return out

ensemble_model = EnsembleModel()

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the mos

In [ ]:
train_acc_ensemble, val_acc_ensemble, epochs_ensemble = train(ensemble_model, train_loader, val_loader, num_epochs=8)
plot_curve(train_acc_ensemble, epochs_ensemble, "Training")
plot_curve(val_acc_ensemble, epochs_ensemble, "Validation")

Training Started...
Epoch 0: Train acc: 0.6521666666666667 |Validation acc: 0.6115
Epoch 1: Train acc: 0.7405 |Validation acc: 0.664
Epoch 2: Train acc: 0.7555 |Validation acc: 0.6785
Epoch 3: Train acc: 0.8216666666666667 |Validation acc: 0.6895
Epoch 4: Train acc: 0.8508333333333333 |Validation acc: 0.7075


KeyboardInterrupt: ignored

In [ ]:
# Define ensemble model using stacking
class StackingEnsemble(nn.Module):
    def __init__(self, models, meta_model):
        super(StackingEnsemble, self).__init__()
        self.models = models
        self.meta_model = meta_model
        
    def forward(self, x):
        outputs = []
        for model in self.models:
            outputs.append(model(x))
        stacked_output = torch.cat(outputs, dim=1)
        output = self.meta_model(stacked_output)
        return output
    
# Define ensemble model using regular averaging
class AveragingEnsemble(nn.Module):
    def __init__(self, models):
        super(AveragingEnsemble, self).__init__()
        self.models = models
    
    def forward(self, x):
        outputs = []
        for model in self.models:
            outputs.append(model(x))
        output = torch.mean(torch.stack(outputs), dim=0)
        return output

# Create stacking ensemble model
meta_model = nn.Sequential(
    nn.Linear(1536, 512),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(512, 5)
)
stacking_ensemble = StackingEnsemble(models, meta_model)

# Create averaging ensemble model
averaging_ensemble = AveragingEnsemble(models)